In [1]:
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import pandas as pd
from pyspark.sql.functions import mean
from matplotlib import pyplot as plt
from pyspark.sql.functions import when

def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark


from pyspark import SparkFiles
spark = init_spark()
sp = None


sp = spark.read.csv("ratings.csv", header=True)

# Book Dataset Cleaning 


In [2]:
#load book dataset
bk = spark.read.csv("books.csv", header=True)
print("Total Number of Books",bk.count())
bk.show(1)

Total Number of Books 10000
+-------+-----------------+------------+-------+-----------+---------+-----------------+---------------+-------------------------+----------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+--------------------+--------------------+
|book_id|goodreads_book_id|best_book_id|work_id|books_count|     isbn|           isbn13|        authors|original_publication_year|  original_title|               title|language_code|average_rating|ratings_count|work_ratings_count|work_text_reviews_count|ratings_1|ratings_2|ratings_3|ratings_4|ratings_5|           image_url|     small_image_url|
+-------+-----------------+------------+-------+-----------+---------+-----------------+---------------+-------------------------+----------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+----

In [3]:
# remove book with <=100 ratings only 
bk = bk.withColumn("ratings_count",bk.ratings_count.cast('int'))
bk = bk.where('ratings_count > 100')
bk.select('ratings_count').orderBy('ratings_count').show()
bk.count()

+-------------+
|ratings_count|
+-------------+
|         2716|
|         2773|
|         3200|
|         3427|
|         3508|
|         3799|
|         4225|
|         4281|
|         4513|
|         4528|
|         4713|
|         4754|
|         4769|
|         4842|
|         4886|
|         4911|
|         4961|
|         5012|
|         5017|
|         5044|
+-------------+
only showing top 20 rows



9998

In [4]:
# remove book publised in year <= 1000 
bk = bk.withColumn("original_publication_year",bk.original_publication_year.cast('int'))
bk = bk.where('original_publication_year > 1000')
bk.select('original_publication_year').orderBy('original_publication_year').show()
bk.count()

+-------------------------+
|original_publication_year|
+-------------------------+
|                     1120|
|                     1273|
|                     1308|
|                     1320|
|                     1320|
|                     1320|
|                     1353|
|                     1380|
|                     1390|
|                     1390|
|                     1418|
|                     1485|
|                     1513|
|                     1516|
|                     1519|
|                     1532|
|                     1550|
|                     1554|
|                     1563|
|                     1589|
+-------------------------+
only showing top 20 rows



9939

In [5]:
#check for null authour
bk.filter(bk.authors.isNull()).select('authors').show()

+-------+
|authors|
+-------+
+-------+



In [6]:
#replace null langauge-code

bk.count()
bk = bk.withColumn("language_code",bk.language_code.cast('string'))

bk.groupBy('language_code').count().select('language_code','count').show(28)

#check null vlaues
bk.filter(bk.language_code.isNull()).select('language_code').count()

#replace null by code 'unknown' - non popular languages 
bk = bk.withColumn("language_code",when(bk.language_code.isNull(),"unknown").otherwise(bk.language_code))

bk.groupBy('language_code').count().select('language_code','count').show(28)


+-------------+-----+
|language_code|count|
+-------------+-----+
|          fre|   25|
|           en|    4|
|        en-CA|   58|
|          rus|    1|
|          ind|   21|
|          per|    7|
|         null| 1077|
|          nor|    3|
|          pol|    6|
|          vie|    1|
|          ara|   62|
|          por|    6|
|          swe|    1|
|          mul|    1|
|          eng| 6300|
|          jpn|    7|
|           nl|    1|
|          dan|    3|
|        en-GB|  256|
|          fil|    2|
|          tur|    1|
|          rum|    1|
|          ita|    2|
|        en-US| 2060|
|          spa|   20|
|          ger|   13|
+-------------+-----+

+-------------+-----+
|language_code|count|
+-------------+-----+
|          fre|   25|
|           en|    4|
|        en-CA|   58|
|          rus|    1|
|          ind|   21|
|      unknown| 1077|
|          per|    7|
|          nor|    3|
|          pol|    6|
|          vie|    1|
|          ara|   62|
|          por|    6|
|        

In [7]:
#check null title & original title

bk = bk.withColumn("title",bk.title.cast('string'))
print("Null title",bk.filter(bk.title.isNull()).count())

bk = bk.withColumn("original_title",bk.original_title.cast('string'))
print("Null original_title",bk.filter(bk.original_title.isNull()).count())

bk.filter(bk.original_title.isNotNull()).select('title','original_title').show()

#column original title will be discarded and title column used only (since only a title can match other books and not the original title)




Null title 0
Null original_title 568
+--------------------+--------------------+
|               title|      original_title|
+--------------------+--------------------+
|The Hunger Games ...|    The Hunger Games|
|Harry Potter and ...|Harry Potter and ...|
|Twilight (Twiligh...|            Twilight|
|To Kill a Mocking...|To Kill a Mocking...|
|    The Great Gatsby|    The Great Gatsby|
|The Fault in Our ...|The Fault in Our ...|
|          The Hobbit|The Hobbit or The...|
|The Catcher in th...|The Catcher in th...|
|Angels & Demons  ...|    Angels & Demons |
| Pride and Prejudice| Pride and Prejudice|
|     The Kite Runner|    The Kite Runner |
|Divergent (Diverg...|           Divergent|
|                1984|Nineteen Eighty-Four|
|         Animal Farm|Animal Farm: A Fa...|
|The Diary of a Yo...|Het Achterhuis: D...|
|The Girl with the...|Män som hatar kvi...|
|Catching Fire (Th...|       Catching Fire|
|Harry Potter and ...|Harry Potter and ...|
|The Fellowship of...| The Fellowship o

In [8]:
#discard un-required columns

bk = bk.select('book_id','authors','original_publication_year','title','language_code','average_rating','ratings_count','ratings_1','ratings_2','ratings_3','ratings_4','ratings_5')
print("Number of Samples after cleaning: ",bk.count())
bk.select('book_id','authors','original_publication_year','title','language_code','average_rating','ratings_count').show()
bk.select('ratings_1','ratings_2','ratings_3','ratings_4','ratings_5').show()

Number of Samples after cleaning:  9939
+-------+--------------------+-------------------------+--------------------+-------------+--------------+-------------+
|book_id|             authors|original_publication_year|               title|language_code|average_rating|ratings_count|
+-------+--------------------+-------------------------+--------------------+-------------+--------------+-------------+
|      1|     Suzanne Collins|                     2008|The Hunger Games ...|          eng|          4.34|      4780653|
|      2|J.K. Rowling, Mar...|                     1997|Harry Potter and ...|          eng|          4.44|      4602479|
|      3|     Stephenie Meyer|                     2005|Twilight (Twiligh...|        en-US|          3.57|      3866839|
|      4|          Harper Lee|                     1960|To Kill a Mocking...|          eng|          4.25|      3198671|
|      5| F. Scott Fitzgerald|                     1925|    The Great Gatsby|          eng|          3.89|      2

# Clean Tags Dataset


In [9]:
#load dataset
tags = spark.read.csv("book_tags.csv", header=True)
tags_name = spark.read.csv("tags.csv", header=True)

print(tags.count())
print(tags_name.count())


tags = tags.join(tags_name, ['tag_id'], 'left_outer')
tags.show(5)

999912
34252
+------+-----------------+------+-----------------+
|tag_id|goodreads_book_id| count|         tag_name|
+------+-----------------+------+-----------------+
| 30574|                1|167697|          to-read|
| 11305|                1| 37174|          fantasy|
| 11557|                1| 34173|        favorites|
|  8717|                1| 12986|currently-reading|
| 33114|                1| 12716|      young-adult|
+------+-----------------+------+-----------------+
only showing top 5 rows



In [10]:
#remove tags that has < 100 books with that label 

tags = tags.withColumnRenamed("count","count_")
tags = tags.withColumn("count_",tags.count_.cast('int'))

count_r = tags.groupBy('tag_id').count()
print("Redundant tags",count_r.where('count < 100').count())

tags = tags.join(count_r, ['tag_id'], 'left_outer')

# filter tags

tags.show(5)

tags = tags.where('count >= 100')

tags.show(5)
tags.count()


Redundant tags 33143
+------+-----------------+------+------------+-----+
|tag_id|goodreads_book_id|count_|    tag_name|count|
+------+-----------------+------+------------+-----+
| 10096|           714711|     1|e-books-read|    1|
| 10351|            10115|     5|  egyptology|    4|
| 10351|           188230|    22|  egyptology|    4|
| 10351|           481446|     8|  egyptology|    4|
| 10351|          2653622|     9|  egyptology|    4|
+------+-----------------+------+------------+-----+
only showing top 5 rows

+------+-----------------+------+--------+-----+
|tag_id|goodreads_book_id|count_|tag_name|count|
+------+-----------------+------+--------+-----+
| 18130|               25|    21|    life|  412|
| 18130|               98|     3|    life|  412|
| 18130|              117|     8|    life|  412|
| 18130|              446|     4|    life|  412|
| 18130|              629|    57|    life|  412|
+------+-----------------+------+--------+-----+
only showing top 5 rows



813611

In [11]:
#remove tags of books which was labeled by < 100 users (e.g less than 100 out of >100 000 users label a book with "action"
#tag means the book is not an action book and the tag should be remove)

tags = tags.where('count_ > 100')
tags.show(5)
tags.count()

+------+-----------------+------+----------+-----+
|tag_id|goodreads_book_id|count_|  tag_name|count|
+------+-----------------+------+----------+-----+
| 18130|             6900|   133|      life|  412|
|  2136|            14384|   139|alpha-male|  588|
|  2136|            35729|   160|alpha-male|  588|
|  2136|            42899|   238|alpha-male|  588|
|  2136|            42900|   160|alpha-male|  588|
+------+-----------------+------+----------+-----+
only showing top 5 rows



111740

In [12]:
print("Number of distinct tags: ",tags.select('tag_name').distinct().count())

tags = tags.select('goodreads_book_id','tag_id','tag_name')

tags.show(5)

Number of distinct tags:  763
+-----------------+------+----------+
|goodreads_book_id|tag_id|  tag_name|
+-----------------+------+----------+
|             6900| 18130|      life|
|            14384|  2136|alpha-male|
|            35729|  2136|alpha-male|
|            42899|  2136|alpha-male|
|            42900|  2136|alpha-male|
+-----------------+------+----------+
only showing top 5 rows



In [13]:
#save cleaned data

bk.write.csv("book_clean.csv",header=True)

tags.write.csv("book-tag_clean.csv",header=True)